In [1]:
import os
import ROOT as r
dict={}
trees=["NOMINAL"]
with open("../../sys_jet_trees.txt","r") as file:
    line=file.readline()
    while line:
        trees.append(line.rstrip('\n'))
        line=file.readline()

samples=["Signal_Sherpa","Signal_PoPy","VV"]

Welcome to JupyROOT 6.22/00


In [3]:


# FILLING DICTIONARY WITH SYS-RELATED TREES 

### the dictionary will have the following structure "systematic_name":[yield]

for sam in samples:
    for sys in trees[1:]:
        file=r.TFile.Open(sam+sys+".root")
        sys_yield=file.Get("ratio_lpt_tpt_cuts_ptl")
        dict[sam+sys]=sys_yield.Integral(0,50)
        file.Close()



Welcome to JupyROOT 6.22/00


In [2]:
#NOW ADDING THE WEIGHT-BASED SYSTEMATICS        
histograms=[]
for sys in trees[:1]:
    file=r.TFile.Open("Signal_Sherpa"+sys+".root")
    histos=file.GetListOfKeys()
    for i in histos:
        name=i.GetName()
        if ("cuts" in name) and ("ptl" in name) and ("yield" in name):
            histograms.append(i.GetName()[len("yield_sys_cuts_ptl_"):])


for sam in samples:
    for sys in trees[:1]:
        file=r.TFile.Open(sam+sys+".root")
        for h in histograms:
            sys_yield=file.Get("yield_sys_cuts_ptl_"+h)
            dict[sam+sys+h]=sys_yield.Integral(0,50)
        sys_yield=file.Get("ratio_lpt_tpt_cuts_ptl")
        dict[sam+sys+"nominal"]=sys_yield.Integral(0,50)
        file.Close()

In [3]:
#The structure of the dictionary is... "name_of_sys":[Sherpa,PoPy,VV]
final_dict={}
for i in histograms+trees[1:]:
    final_dict[i]=[0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict[i][0]=dict[j]
        if (i in j) and ("PoPy" in j):
            final_dict[i][1]=dict[j]
        if (i in j) and ("VV" in j):
            final_dict[i][2]=dict[j]
            
#TAKING DOWN AND UP SYSTEMATICS TO THE BOTTOM AND THE TOP OF THE DICTIONARY RESPECTIVELY
up={}
down={}

a_keys=[i for i in final_dict]

for i in a_keys:
    if "1down" in i:
        down.update({i:final_dict[i]})
        final_dict.pop(i)

final_dict.update(down)


In [4]:
# EXPORTING THE RESULTS TO A CSV FILE
import pandas as pd
df=pd.DataFrame.from_dict(final_dict, orient='index',
                       columns=["Sherpa","PoPy","VV"])


df.to_csv('Uncertainties.csv')



In [6]:
final_dict["jet_JET_JvtEfficiency_1down_central_jets_global_effSF_JVT"]

[321031.81799316406, 335466.77111816406, 6791.089002609253]